### Search of runtime errors
This notebook has exactly the same code as train.py except for the argument parser

In [1]:
from __future__ import print_function
from easydict import EasyDict as edict
from lib.cfgs import c as dcfgs
import lib.cfgs as cfgs
import os
os.environ['JOBLIB_TEMP_FOLDER']=dcfgs.shm
import argparse
os.environ['GLOG_minloglevel'] = '3'
import os.path as osp
import pickle
import sys
from multiprocessing import Process, Queue

import matplotlib.pyplot as plt
import numpy as np
from IPython import embed

from lib.decompose import *
from lib.net import Net, load_layer, caffe_test
from lib.utils import *
from lib.worker import Worker
import google.protobuf.text_format # added to fix missing protobuf properties -by Mario
sys.path.insert(0, osp.dirname('__file__')+'/lib')

no lighting pack


In [ ]:
def step0(pt, model):
    net = Net(pt, model=model, noTF=1)  # lib/net.Net instantiate the NetBuilder -by Mario
    WPQ, pt, model = net.preprocess_resnet() # WPQ stores pruned values, which will be later saved to the caffemodel -by Mario
    return {"WPQ": WPQ, "pt": pt, "model": model}

def step1(pt, model, WPQ, check_exist=False):
    print(pt)
    net = Net(pt, model, noTF=1)
    model = net.finalmodel(WPQ) # loads weights into the caffemodel - by Mario
    if 1:#TODO: Consider adding a configuration paramter to cfgs.py in order to control whether or not to prune the last conv layer -by Mario
        convs = net.convs
        redprint("including last conv layer!")
    else:
        convs = net.convs[:-1]
        redprint("ignoring last conv layer!")
    if dcfgs.dic.option == 1:
        sums = net.type2names('Eltwise')[:-1]
        newsums = []
        for i in sums:
            if not i.endswith('block8_sum'):
                newsums.append(i)
        newconvs = []
        for i in convs:
            if i.endswith('_proj'):
                newconvs.insert(0,i)
            else:
                newconvs.append(i)
        convs = newsums + newconvs
    else:
        convs += net.type2names('Eltwise')[:-1]
    if dcfgs.dic.fitfc:
        convs += net.type2names('InnerProduct')
    if dcfgs.model in [cfgs.Models.xception,cfgs.Models.resnet]:
        for i in net.bns:
            if 'branch1' in i:
                convs += [i]
    net.freeze_images(check_exist=check_exist, convs=convs)
    return {"model":model}

def combine():
    net = Net(dcfgs.prototxt, dcfgs.weights)
    net.combineHP()

def splitrelu():
    net = Net(dcfgs.prototxt, model=dcfgs.weights)
    print(net.seperateConvReLU())

def addbn(pt='../resnet-cifar10-caffe/resnet-56/prb_mem_bn_trainval.prototxt', model="../resnet-cifar10-caffe/resnet-56/snapshot/prb_VH_bn__iter_64000.caffemodel"):
    worker=Worker()
    def ad(pt, model):
        net = Net(pt, model=model, noTF=1)
        #net.computation()
        pt, WPQ = net.add_bn()
        return {'new_pt': pt, 'model':model, 'WPQ':WPQ}
    outs = worker.do(ad, pt=pt, model=model)
    worker.do(stepend, **outs)
    #stepend(**outs)

def compute(pt='../resnet-cifar10-caffe/resnet-56/trainval.prototxt', model="../resnet-cifar10-caffe/resnet-56/snapshot/_iter_64000.caffemodel"):
    net = Net(pt, model=model, noTF=1)
    net.computation()

In [2]:
# main
cfgs.set_nBatches(dcfgs.nBatches)
dcfgs.dic.option=1
print("nBatches ", dcfgs.nBatches)
print("nBatches_fc ", dcfgs.nBatches_fc)

DEBUG_Mario = 1


# c3()

nBatches  500
nBatches_fc  500


# Function call: c3()

In [ ]:
"""
def c3(pt=cfgs.vgg.model,model=cfgs.vgg.weights):
TODO: Consider changing cfgs.vgg.model and cfgs.vgg.weights 
      (paths to the .prototxt and .caffemodel files) 
      for a generic model reference -by Mario
"""
pt=cfgs.vgg.model
model=cfgs.vgg.weights

dcfgs.splitconvrelu=True
cfgs.accname='accuracy@5' # name of layer in the prototxt -by Mario

In [3]:
#worker = Worker()
#outputs = worker.do(step0, pt=pt, model=model)
"""
step0 execution
def step0(pt, model)
"""
pt=cfgs.vgg.model
model=cfgs.vgg.weights

net = Net(pt, model=model, noTF=1)  # lib/net.Net instantiate the NetBuilder -by Mario
WPQ, pt, model = net.preprocess_resnet() # WPQ stores pruned values, which will be later saved to the caffemodel -by Mario
outputs = {"WPQ": WPQ, "pt": pt, "model": model}
 
printstage("freeze")
pt = outputs['pt'] 

"""end of step0 execution"""

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
stage0 freeze
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


'end of step0 execution'

In [4]:
outputs

{'WPQ': {}, 'model': 'temp/bn_vgg.caffemodel', 'pt': 'temp/bn_vgg.prototxt'}

In [5]:
#outputs = worker.do(step1,**outputs)
"""
step1 execution
def step1(pt, model, WPQ, check_exist=False)
"""
check_exist = False
print(pt)
net = Net(pt, model, noTF=1) # re-instantiate the net object with new pt and model 
model = net.finalmodel(WPQ) # loads weights into the caffemodel - by Mario

if 1:#TODO: Consider adding a configuration paramter to cfgs.py in order to control whether or not to prune the last conv layer -by Mario
    convs = net.convs
    redprint("including last conv layer!")
else:
    convs = net.convs[:-1]
    redprint("ignoring last conv layer!")

print(convs)

if dcfgs.dic.option == 1:
    if DEBUG_Mario: print("this line executed because dcfgs.dic.option is 1")
    sums = net.type2names('Eltwise')[:-1]
    newsums = []
    for i in sums:
        if not i.endswith('block8_sum'):
            newsums.append(i)
    newconvs = []
    for i in convs:
        if i.endswith('_proj'):
            newconvs.insert(0,i)
        else:
            newconvs.append(i)
    convs = newsums + newconvs
else:
    convs += net.type2names('Eltwise')[:-1]
    
if dcfgs.dic.fitfc:
    convs += net.type2names('InnerProduct')
if dcfgs.model in [cfgs.Models.xception,cfgs.Models.resnet]:
    for i in net.bns:
        if 'branch1' in i:
            convs += [i] 

temp/bn_vgg.prototxt
including last conv layer!
['conv1_1', 'conv1_2', 'conv2_1', 'conv2_2', 'conv3_1', 'conv3_2', 'conv3_3', 'conv4_1', 'conv4_2', 'conv4_3', 'conv5_1', 'conv5_2', 'conv5_3']


---          
## Function call:  net.freeze_images(check_exist=check_exist, convs=convs)



In [6]:
"""start of net.freeze_images"""
# def freeze_images(self, check_exist=False, convs=None, **kwargs):
if cfgs.layer:
    frozen = net._frozen_layer
    if check_exist: # this if-condition should be implemented
        pass
    else:
        pass
else:
    frozen = net._frozen
    if check_exist:
            if osp.exist(frozen):
                print("Exist", frozen)
print("froze: ", frozen)

if convs is None:
    convs = self.type2names()
if cfgs.layer:
    #feats_dict, points_dict = net.extract_layers(convs)
    print("extract_layers (single-layer pruning)")
else:
    print("extract_features (full pruning)")
    

temp/frozen500.pickle


 
### Function call: feats_dict, points_dict = net.extract_features(names=convs, save=1)
  

In [8]:
#def extract_features(self, names=[], nBatches=None, points_dict=None, save=False):
"""start of net.extract_features"""
nBatches = None
names = convs
points_dict=None
save = 1

nBatches = dcfgs.nBatches
nPointsPerLayer = dcfgs.nPointsPerLayer

if not isinstance(names, list):
    names = [names]
inner = False

if len(names)==1:
    print("this code only executes for FC layers")
    for top in self.innerproduct:
        if names[0] in self.bottom_names[top]:
            inner = True
            nBatches = dcfgs.nBatches_fc
            break

DEBUG = False

pads = dict()
shapes = dict()
feats_dict = dict()

def set_points_dict(name, data):
    assert name not in points_dict
    points_dict[name] = data

dcfgs.data = cfgs.Data.lmdb  # I think this disables the use of Data.pro type of data -by Mario
if save:
    if points_dict is None:
        frozen_points = False
        points_dict = dict()
        #if 0 and self._mem:
        #   self.usexyz()
        set_points_dict("nPointsPerLayer", nPointsPerLayer)
        set_points_dict("nBatches", nBatches)
    else:
        frozen_points = True
        if nPointsPerLayer != points_dict["nPointsPerLayer"] or nBatches != points_dict["nBatches"]:
            print("overwriting nPointsPerLayer, nBatches with frozen_points")

        nPointsPerLayer = points_dict["nPointsPerLayer"]
        nBatches = points_dict["nBatches"]
        
 

assert len(names) > 0 

nPicsPerBatch = net.blobs_num(names[0])
nFeatsPerBatch = nPointsPerLayer  * nPicsPerBatch

print("run for", dcfgs.nBatches, "batches", "nFeatsPerBatch", nFeatsPerBatch)
nFeats = nFeatsPerBatch * nBatches

print("points_dict: ", points_dict)
print("feats_dict:", feats_dict  )

run for 500 batches nFeatsPerBatch 100
points_dict:  {'nPointsPerLayer': 10, 'nBatches': 500}
feats_dict: {}


In [9]:
print("points_dict: ", points_dict)
print("feats_dict:", feats_dict  )
print("nBatches", dcfgs.nBatches) #conig defautls
print("nBatches_fc", dcfgs.nBatches_fc) # config defaults
print("nPicsPerBatch", nPicsPerBatch) # defined in the prototxt and stured in net.num
print("nPointsPerLayer ", nPointsPerLayer) # config defaults (this shold be call nPointsPerChannelofaLayer)
print("nFeatsPerBatch",nFeatsPerBatch )# =  nPointsPerLayer  * nPicsPerBatch
print("nFeats", nFeats) # = nFeatsPerBatch * nBatches

points_dict:  {'nPointsPerLayer': 10, 'nBatches': 500}
feats_dict: {}
nBatches 500
nBatches_fc 500
nPicsPerBatch 10
nPointsPerLayer  10
nFeatsPerBatch 100
nFeats 50000


In [10]:
for name in names:

    """avoiding X out of bound"""
    shapes[name] = (net.blobs_height(name), net.blobs_width(name))

    if inner or len(net.blobs_shape(name))==2 or ( shapes[name][0] == 1 and shapes[name][1] == 1):
        print("this condition should not execute or standard run on VGG")
        if 0: print(name)
        chs = net.blobs_channels(name)
        if len(net.blobs_shape(name)) == 4:
            chs*=shapes[name][0]*shapes[name][1]
        feats_dict[name] = np.ndarray(shape=(nPicsPerBatch * dcfgs.nBatches_fc,chs ))
    else:
        feats_dict[name] = np.ndarray(shape=(nFeats, net.blobs_channels(name)))
    print("Extracting", name, feats_dict[name].shape) # for a standard run,  names is a list with the conv layers: name = convs -by Mario

idx = 0
fc_idx = 0
if save:
    if not frozen_points: 
        """there is no frozen points, we have to save the one we will prepare"""
        set_points_dict("data", net.data().shape)
        set_points_dict("label", net.label().shape)

Extracting conv1_1 (50000, 64)
Extracting conv1_2 (50000, 64)
Extracting conv2_1 (50000, 128)
Extracting conv2_2 (50000, 128)
Extracting conv3_1 (50000, 256)
Extracting conv3_2 (50000, 256)
Extracting conv3_3 (50000, 256)
Extracting conv4_1 (50000, 512)
Extracting conv4_2 (50000, 512)
Extracting conv4_3 (50000, 512)
Extracting conv5_1 (50000, 512)
Extracting conv5_2 (50000, 512)
Extracting conv5_3 (50000, 512)


In [13]:
print("points_dic was updated: ", points_dict)

points_dic was updated:  {'nPointsPerLayer': 10, 'nBatches': 500, 'data': (10, 3, 224, 224), 'label': (10,)}


In [18]:
# for suding layes' structure
this_layer = 'conv1_1'
print("width and height of %s: (%d,%d) | net.shapes[layer]: " % (this_layer, shapes['conv1_1'][0], shapes['conv1_1'][1]))
print("width and height of %s: (%d,%d) | net.blobs_x(layer): " % ( this_layer ,net.blobs_width(this_layer), net.blobs_height(this_layer) ))

width and height of conv1_1: (224,224) | net.shapes[layer]: 
width and height of conv1_1: (224,224) | net.blobs_x(layer): 


In [19]:
# feats_dict has now "key" values. However has not been initialized, it holds zeros
feats_dict['conv1_1'][49999][63]

0.0

In [21]:
# since we have no executed forward pass, is all zeros
feat = net.blobs_data('conv1_1')
print(feat.shape)
i_feat = np.nonzero(feat)
print(i_feat)

(10, 64, 224, 224)
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


In [22]:
DEBUG_Mario = 1

runforn = dcfgs.nBatches_fc if dcfgs.dic.fitfc else dcfgs.nBatches
runforn = 2 # Override runforn for testing
for batch in range(runforn):
    if save:
        """ first copy the data layer content into the points_dict"""
        if not frozen_points:
            net.forward()
            #print("net forward, frozen_point is false")
            set_points_dict((batch, 0), net.data().copy())
            set_points_dict((batch, 1), net.label().copy())
        else:
            net.net.set_input_arrays(points_dict[(batch, 0)], points_dict[(batch, 1)])
            #print("net forward, frozen_point is true")
            net.forward()
    else:
        net.forward()
        #print("net forward, no save")
     
    """ then get the  activation of each layer """
    for name in names:
        # pad = pads[name]
        shape = shapes[name]
        feat = net.blobs_data(name)
        print("batch:", batch, "layer: ", name, "feature map: ", net.blobs_shape(name))
        
        if inner or len(net.blobs_shape(name))==2 or (shape[0] == 1 and shape[1] == 1):
            print("deal with FC layer(?) name: ", name)
            feats_dict[name][fc_idx:(fc_idx + nPicsPerBatch)] = feat.reshape((net.num, -1))
            continue
        if batch >= dcfgs.nBatches and name in net.convs:
            print("prevent the code bellow from executing if we exceed nBatchs(?)" )
            continue 
        # TODO!!! different patch for different image per batch
        if save:
            if not frozen_points or (batch, name, "randx") not in points_dict:
                #embed()
                """ and create random coordinates for the sampling process """
                randx = np.random.randint(0, shape[0]-0, nPointsPerLayer)
                randy = np.random.randint(0, shape[1]-0, nPointsPerLayer)
              
                if dcfgs.dic.option == cfgs.pruning_options.resnet:
                    if DEBUG_Mario: print("this line executed when dcfgs.dic.option is resnet")
                    branchrandxy = None
                    branch1name = '_branch1'
                    branch2cname = '_branch2c'
                    if name in self.sums:
                        #embed()
                        nextblock = self.sums[self.sums.index(name)+1]
                        nextb1 = nextblock + branch1name
                        if not nextb1 in names:
                            # the previous sum and branch2c will be identical
                            branchrandxy = nextblock + branch2cname
                    elif name in self.bns:
                        if dcfgs.model == cfgs.Models.xception:
                            branchrandxy = 'interstellar' + name.split('bn')[1].split('_')[0] + branch2cname
                        elif dcfgs.model == cfgs.Models.resnet:
                            branchrandxy = 'res' + name.split('bn')[1].split('_')[0] + branch2cname
                            #print("correpondance", branchrandxy)
                    if branchrandxy is not None:
                        if 0: print('pointsdict of', branchrandxy, 'identical with', name)
                        randx = points_dict[(batch, branchrandxy , "randx")]
                        randy = points_dict[(batch, branchrandxy , "randy")]
               
                if name.endswith('_conv1') and dcfgs.dic.option == 1:
                    if DEBUG_Mario: print("this line executed because dcfgs.dic.option is 1")
                    fsums = ['first_conv'] + self.sums
                    blockname = name.partition('_conv1')[0]
                    nextb1  = fsums[fsums.index(blockname+'_sum')-1]
                    branch1name = blockname + '_proj'
                    if branch1name in self.convs:
                        nextb1 = branch1name
                    randx = points_dict[(batch, nextb1 , "randx")]
                    randy = points_dict[(batch, nextb1 , "randy")]
                    
                # add the randome coordinates to the points_dict
                set_points_dict((batch, name, "randx"), randx.copy())
                set_points_dict((batch, name, "randy"), randy.copy())

            else:
                randx = points_dict[(batch, name, "randx")]
                randy = points_dict[(batch, name, "randy")]
        else:
            randx = np.random.randint(0, shape[0]-0, nPointsPerLayer)
            randy = np.random.randint(0, shape[1]-0, nPointsPerLayer)

        for point, x, y in zip(range(nPointsPerLayer), randx, randy):
            """ Finally, use the random coordinates to sample points from each channel in a layer
            (10 points per channel, per each image per  batch). If we have :
                |64 channels in a layer
                |10 images per batch (validation dataset)
                |500 batches (default in the code )
                |
                +-----> Then we will sample 5000 images, and a 10 point per each channel,
                        (50000, 64) shaped array of sampled points 
            """
            i_from = idx+point*nPicsPerBatch
            try:
                feats_dict[name][i_from:(i_from + nPicsPerBatch)] = feat[:,:,x, y].reshape((net.num, -1))
            except:
                 print('total', runforn, 'batch', batch, 'from', i_from, 'to', i_from + nPicsPerBatch)
                 raise Exception("out of bound")
        if DEBUG:
            embed()
    
    idx += nFeatsPerBatch
    fc_idx += nPicsPerBatch

batch: 0 layer:  conv1_1 feature map:  (10, 64, 224, 224)
batch: 0 layer:  conv1_2 feature map:  (10, 64, 224, 224)
batch: 0 layer:  conv2_1 feature map:  (10, 128, 112, 112)
batch: 0 layer:  conv2_2 feature map:  (10, 128, 112, 112)
batch: 0 layer:  conv3_1 feature map:  (10, 256, 56, 56)
batch: 0 layer:  conv3_2 feature map:  (10, 256, 56, 56)
batch: 0 layer:  conv3_3 feature map:  (10, 256, 56, 56)
batch: 0 layer:  conv4_1 feature map:  (10, 512, 28, 28)
batch: 0 layer:  conv4_2 feature map:  (10, 512, 28, 28)
batch: 0 layer:  conv4_3 feature map:  (10, 512, 28, 28)
batch: 0 layer:  conv5_1 feature map:  (10, 512, 14, 14)
batch: 0 layer:  conv5_2 feature map:  (10, 512, 14, 14)
batch: 0 layer:  conv5_3 feature map:  (10, 512, 14, 14)
batch: 1 layer:  conv1_1 feature map:  (10, 64, 224, 224)
batch: 1 layer:  conv1_2 feature map:  (10, 64, 224, 224)
batch: 1 layer:  conv2_1 feature map:  (10, 128, 112, 112)
batch: 1 layer:  conv2_2 feature map:  (10, 128, 112, 112)
batch: 1 layer:  co

--------

In [33]:
# Now the activation (feaure maps) has been initialzied
feat = net.blobs_data('conv1_1')
print("feat.shape: ", feat.shape)
i_feat = np.nonzero(feat)
print("no zero elements of feat: ", i_feat)

print("--------- Feature Map Values ----------" )
channel = feat[9][63]
#channel = feat[10][63] is out of bound because there is only 9 images per batch  (0 indexing)
#channel = feat[9][64] is out of bound because there is only 64 channels in conv1_1 (0 indexing)
print("channel shape: ", channel.shape)
#print(channel)

if runforn == 2:
    # feats_dict has been updated with new random samples 
    # however, we only ran for 2 batches so a lot of entries are still zero
    print("--------- Feature Samples Dictionary ----------" )
    print("feats_dict['conv1_1'].shape: ", feats_dict['conv1_1'].shape)
    print("feats_dict['conv1_1'][0][63] was modified: ", feats_dict['conv1_1'][0][63])
    print("feats_dict['conv1_1'][49999][63] was not: ",feats_dict['conv1_1'][49999][63])

feat.shape:  (10, 64, 224, 224)
no zero elements of feat:  (array([0, 0, 0, ..., 9, 9, 9]), array([ 0,  0,  0, ..., 63, 63, 63]), array([  0,   0,   0, ..., 223, 223, 223]), array([  0,   1,   2, ..., 221, 222, 223]))
--------- Feature Map Values ----------
channel shape:  (224, 224)
--------- Feature Samples Dictionary ----------
feats_dict['conv1_1'].shape:  (50000, 64)
feats_dict['conv1_1'][0][63] was modified:  2.02482795715
feats_dict['conv1_1'][49999][63] was not:  0.0


In [34]:
print("feats_dict:", feats_dict)

feats_dict: {'conv4_1': array([[ -404.42437744,   282.1819458 ,  -179.07402039, ...,
        -2077.56469727,  1370.03405762, -1473.65600586],
       [    2.15681624,   -23.00135612,  -101.84294128, ...,
           57.3445816 ,    64.9425354 ,   199.47135925],
       [-1107.15710449,  -572.66339111,  1992.01806641, ...,
         -520.57092285,   805.49700928,   231.48554993],
       ..., 
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ]]), 'conv1_2': array([[ 101.15026093,   58.07224655,  -63.68658447, ...,   50.85097122,
         -33.3972702 ,  -17.4185009 ],
       [  84.10009003,  293.05969238,  639.12677002, ...,   76.52697754,
        -229.31529236,   57.52775574],
       [ -76.34429

In [39]:
# points_dict was also modified. We added the random corrdinates to sample each batch
# we alse added the data layer values a
print("data layer values for batch 1 (shape):", points_dict[(0,0)].shape)
print("data layer labels for batch 1 (shape):", points_dict[(0,1)].shape)

 
print("batch 1, conv1_1, randx: ",points_dict[(1,'conv1_1','randx')])
print("batch 1, conv1_1, randy: ",points_dict[(1,'conv1_1','randy')])

data layer values for batch 1 (shape): (10, 3, 224, 224)
data layer labels for batch 1 (shape): (10,)
batch 1, conv1_1, randx:  [131  95  19 223  70 158 167 103   0 120]
batch 1, conv1_1, randy:  [ 67  50   9   0 215  12  59 124 170  41]


In [54]:
""" examine feat """
#my_dict = feats_dict
feat = net.blobs_data('conv1_1')

print("for conv1_1, feat.shape is :",feat.shape) 
for point, x, y in zip(range(nPointsPerLayer), randx, randy):
# use the random coordinates to sample from feats (activation blobs?) into the feats_dict    
    i_from = idx+point*nPicsPerBatch
    """NOTE:
            For each  image , we extract 10 points points from each layer channel"""
    a = feat[:,:,x, y].reshape((net.num, -1)) 
     
    if i_from == 0:
        print("sample feature's array shape: ", a.shape)
        print("--------")
    print("index :", i_from)  
    print("sampled features (channel 1):", a[:,63]) 

for conv1_1, feat.shape is : (10, 64, 224, 224)
sample feature's array shape:  (10, 64)
--------
index : 0
sampled features (channel 1): [ 14.36517906   5.88363218 -10.3242197  -10.01940441   0.73159975
  -4.55395794   6.04811811  -1.29261112  10.54183674  -9.22943974]
index : 10
sampled features (channel 1): [-26.24725342   9.58125591 -10.24522495 -12.58506775 -66.23661804
  -8.05879498   3.82703972  -1.3037535   10.35839558 -10.05900192]
index : 20
sampled features (channel 1): [-28.89755058  10.39723682 -10.46149349 -10.00550842  43.90003586
  -5.3795166    0.75988054  -8.75227547  10.44703293 -10.49456596]
index : 30
sampled features (channel 1): [-14.58647442   9.30111122 -10.41462612 -10.37274647 -17.75824165
  -8.36483383   1.99208069 -48.1091423   10.49388504  -9.47496319]
index : 40
sampled features (channel 1): [ 12.2435379    9.41791153 -10.42825127 -10.96895981  20.88611031
  -5.66060257   1.93174982  12.44420719  10.21056461  -9.79193211]
index : 50
sampled features (chann

--------

In [57]:
dcfgs.data = cfgs.Data.lmdb # this simple resets the config (why?)
net.clr_acc()
if save:
    if frozen_points:
        if points_dict is not None:
            print("return feats_dict, points_dict")
            #return feats_dict, points_dict
        #return feats_dict
        print("[points_dict is None] return feats_dict")
    else:
        print("[fronze_points is False] return feats_dict, points_dict")
        #return feats_dict, points_dict
else:
    print("[save is false] return feats_dict")
    #return feats_dict
"""end of net.extract_features"""

Acc 100.000
[fronze_points is False] return feats_dict, points_dict


'end of net.extract_features'

### End of function call: feats_dict, points_dict = net.extract_features(names=convs, save=1)

In [59]:
data_layer = net.data_layer
if len(net.net_param_layer(data_layer)) == 2: # this call reaaches the NetBuilder for
    if DEBUG_Mario: print("something happened here")
    net.net_param.net.layer.remove(net.get_layer(data_layer))

# we will prepare the data laye to run with MemoryData type
i = net.get_layer(data_layer)
i.type = "MemoryData"
i.memory_data_param.batch_size = points_dict['data'][0]
i.memory_data_param.channels = points_dict['data'][1]
i.memory_data_param.height = points_dict['data'][2]
i.memory_data_param.width = points_dict['data'][3]
i.ClearField("transform_param")
i.ClearField("data_param")
i.ClearField("include")

print("wrote memory data layer to", net.save_pt(prefix="mem"))
print("freezing imgs to", frozen)

if cfgs.layer:
    if DEBUG_Mario: print("operation on single-layer pruning") 
    
    def subfile(filename):
        return osp.join(frozen, filename)
    
    with open(subfile(net._points_dict_name), 'wb') as f:
        print("dumping points_dict")
        pickle.dump(points_dict, f, protocol=net._protocol)
    if DEBUG_Mario: print("Will dump sampled features (feats_dict) of each layer independantly") 
    for conv in convs:
        with open(subfile(conv), 'wb') as f:
            print("dumping "+conv)
            pickle.dump(feats_dict[conv], f, protocol=net._protocol)

else:
    with open(frozen, 'wb') as f:
        pickle.dump([feats_dict, points_dict], f, protocol=net._protocol)

#return frozen
print(frozen)

something happened here
wrote memory data layer to temp/mem_bn_vgg.prototxt
freezing imgs to temp/frozen500.pickle
temp/frozen500.pickle


## End of function call:  net.freeze_images(check_exist=check_exist, convs=convs)

In [60]:
print("return {'model': model}")
outputs = {"model":model}
"""end of step1 execution"""

return {'model': model}


'end of step1 execution'

In [66]:
if DEBUG_Mario: print("output returned from step 1: ", outputs) 
printstage("speed ", dcfgs.dic.keep)
outputs['pt'] = mem_pt(pt) # update pt to point to the new prototxt (with data layer of  type MemoryData)
if DEBUG_Mario: print("output modified" ,outputs)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
stage3 speed 3.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
perform pruning: solve()


### Fuction call:  outputs = solve(**outputs)

In [ ]:
"""
solve execution
def solve(pt, model)
"""

# what is the value of pt? and model?
# check the excution of worker.do() to find out how it extracts parameters from **outputs
pt = outputs['pt'] #?
model = outputs['model'] #?

net = Net(pt, model=model)
net.load_frozen() # this method can load images from memory if we pass a feats_dic. For what?

WPQ, new_pt = net.R3()
print("return {'WPQ': WPQ, 'new_pt': new_pt}")
"""end of solve execution"""

### End of function call: outputs = solve(**outputs)

In [ ]:
outputs = {"WPQ": WPQ, "new_pt": new_pt}
if DEBUG_Mario: print("output returned from solve : ", outputs) 
printstage("saving")

### Fuction call:  outputs = stepend(model=model, **outputs)

In [ ]:
"""
stepned execution
def stepend(new_pt, model, WPQ):
"""
# what is the value of new_pt? and WPQ?
# check the excution of worker.do() to find out how it extracts parameters from **outputs
new_pt = outputs['new_pt'] #?
WPQ = outputs['WPQ'] #?
model = cfgs.vgg.weights #?

net = Net(new_pt, model=model)
net.WPQ = WPQ
net.finalmodel(save=False) # load weights into the caffemodel -by Mario
net.dis_memory()

#final = net.finalmodel(WPQ, prefix='3r')
new_pt, new_model = net.save(prefix='3c')

print('caffe test -model',new_pt, '-weights',new_model)
print("return {'final': None})

### End of fuction call:  outputs = stepend(model=model, **outputs)

In [ ]:
outputs = {'final': None}
if DEBUG_Mario: print(outputs)

# End of function call: c3()

In [ ]:
"""
def parse_args():
    parser = argparse.ArgumentParser("experiment")
    parser.add_argument('-tf', dest='tf_vis', help='tf devices', default=None, type=str)
    parser.add_argument('-caffe', dest='caffe_vis', help='caffe devices', default=None, type=str)
    parser.add_argument('-action', dest='action', help='action', default='train', type=str)
    attrs = ['dic', 'an', 'res']
    for d in attrs:
        for i in dcfgs[d]:
            parser.add_argument('-'+d+'.'+i, dest=d+'DOT'+i, help=d+'.'+i, default=None,type=str)

    for i in dcfgs:
        if i not in attrs:
            parser.add_argument('-'+i, dest=i, help=i, default=None,type=str)

    args = parser.parse_args()
    if args.tf_vis is not None: cfgs.tf_vis = args.tf_vis
    if args.caffe_vis is not None: cfgs.caffe_vis = args.caffe_vis
    for d in attrs:
        for i in dcfgs[d]:
            att = getattr(args, d+'DOT'+i)
            if att is not None:
                if 0:
                    print(d,i, att)
                dcfgs[d][i]=type(dcfgs[d][i])(att)

    for i in dcfgs:
        if i in attrs:
            continue
        att = getattr(args, i)
        if att is not None:
            dcfgs[i]=type(dcfgs[i])(att)

    dcfgs.Action = args.action
    if args.model is not None:
        netmodel = getattr(cfgs, args.model)
        cfgs.accname = netmodel.accname
        if args.prototxt is None:
            dcfgs.prototxt = netmodel.model
        if args.weights is None:
            dcfgs.weights = netmodel.weights
    return args
"""

### Default code (except arg parser) execution test

In [7]:

"""
if __name__ == '__main__':

    args = parse_args()
    cfgs.set_nBatches(dcfgs.nBatches)

    dcfgs.dic.option=1

    if args.action == cfgs.Action.addbn:
        addbn(pt=dcfgs.prototxt, model=dcfgs.weights)

    elif args.action == cfgs.Action.splitrelu:
        splitrelu()

    elif args.action == cfgs.Action.c3:
        c3()
    elif args.action == cfgs.Action.combine:
        combine()
    else:
        pass
"""

cfgs.set_nBatches(dcfgs.nBatches)
dcfgs.dic.option=1

#desired action is c3(), so ...
c3()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
stage0 freeze
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
temp/bn_vgg.prototxt
including last conv layer!
run for 500 batches nFeatsPerBatch 10
Extracting conv1_1 (5000, 64)
Extracting conv1_2 (5000, 64)
Extracting conv2_1 (5000, 128)
Extracting conv2_2 (5000, 128)
Extracting conv3_1 (5000, 256)
Extracting conv3_2 (5000, 256)
Extracting conv3_3 (5000, 256)
Extracting conv4_1 (5000, 512)
Extracting conv4_2 (5000, 512)
Extracting conv4_3 (5000, 512)
Extracting conv5_1 (5000, 512)
Extracting conv5_2 (5000, 512)
Extracting conv5_3 (5000, 512)
Acc  90.200
wrote memory data layer to temp/mem_bn_vgg.prototxt
freezing imgs to temp/frozen500.pickle
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
stage1 speed3.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
loading imgs from temp/frozen500.pickle
loaded
Extracting X relu1_1 From Y conv1_2 stride 1
Acc  90.200
spatial_decomposition 9.814770460128784
